# Server

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import http.client
import json

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np

In [ ]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge

from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

In [ ]:
import math

## Variables

### Database

In [ ]:
train_request_url = "/pls/apex/kpi_forecast/Data-Transfer/get-model-train"
train_request_count_url = '/pls/apex/kpi_forecast/Data-Transfer/train-request'
kpi_data_url = '/pls/apex/kpi_forecast/Data-Transfer/kpi-data?id="KPI_ID"'
train_data_url = '/pls/apex/kpi_forecast/Data-Transfer/train-data?id="KPI_ID"&v_list="VARS"'

### Local paths

In [ ]:
dataset_main_path = '/content/drive/MyDrive/Colab Notebooks/DatabaseSets'
label_encoders_main_path = '/content/drive/MyDrive/Colab Notebooks/LabelEncoders'
model_saves_main_path = '/content/drive/MyDrive/Colab Notebooks/Models'

## Functions

### Oracle DB

In [ ]:
def getDataApex(endpoint, maxLoop = 999):
  Apex_Host = "apex.oracle.com"
  Data_URL = endpoint.replace(Apex_Host,"")
  Apex_conn = http.client.HTTPSConnection(Apex_Host)
  rows = []

  #Read first 10K row from API
  Apex_conn.request("GET", Data_URL)
  response = Apex_conn.getresponse()
  data = response.read().decode("utf-8")
  data = data.replace('\n', '')

  try:
   data_json = json.loads(data)
  except:
    print('[DB connection error]: response can not converted to JSON')
    return None

  if 'items' in data_json:
    rows = data_json['items']

  if 'hasMore' in data_json and data_json['hasMore']:
    next_url = str(data_json['links'][3]['href']).replace(Apex_Host, "")
  else:
    return rows

  i = 1
  while('hasMore' in data_json and data_json['hasMore'] and i < maxLoop):                      #API's one page can only show 10K rows. It has a link inside for next 10K rows.
    Apex_conn.request("GET", next_url)
    response = Apex_conn.getresponse()
    data = response.read().decode("utf-8")

    data_json = json.loads(data)
    rows += data_json['items']
    next_url = str(data_json['links'][3]['href']).replace(Apex_Host, "")
    i += 1

  Apex_conn.close()

  return rows

In [ ]:
def postApex(endpoint):

  conn = http.client.HTTPSConnection("apex.oracle.com")
  conn.request("POST", endpoint)
  res = conn.getresponse()
  data = res.read()
  return data.decode("utf-8")

### Data Pre-Process

In [ ]:
def data_smote(dataframe, kpi_column_name):
  y = dataframe.pop(kpi_column_name).to_numpy()
  X = dataframe.to_numpy()

  # Data augmentasion
  smote = SMOTE()
  X_resampled, y_resampled = smote.fit_resample(X, y)

  # Örneklem öncesi ve sonrası sınıf dağılımı
  print("Örneklem Öncesi Sınıf Dağılımı:")
  print(y.value_counts())

  print("Örneklem Sonrası Sınıf Dağılımı:")
  print(pd.Series(y_resampled).value_counts())

  return None


In [ ]:
def LSTM_create_dataset(dataset, time_steps=1):
    X, y = [], []

    for i in range(len(dataset)-time_steps):
        X.append(dataset[i:(i+time_steps), 0])
        y.append(dataset[i+time_steps, 0])

    return np.array(X), np.array(y)

In [ ]:
def clean_nulls(dataframe, cleanList = ['?', '-']):

  for chr in cleanList:
    dataframe.replace(chr, pd.NaT, inplace=True)

  print('-------------------------------')
  print(dataframe.isnull().sum())
  print('-------------------------------')
  dataframe = dataframe.dropna()

  return dataframe

In [ ]:
def encodeText(dataframe, request_id=-1): # encode categorial data

  for column in dataframe.columns:
    if dataframe[column].dtype not in [int, float]:
      label_encoder = LabelEncoder()
      clmn = dataframe.pop(column)
      encoded_column = label_encoder.fit_transform(clmn)
      dataframe[f'{column}'] = encoded_column

      with open(f'{label_encoders_main_path}/model_{request_id}_{column}.pkl', 'wb') as f:
        pickle.dump(label_encoder, f)

In [ ]:
def turn_numeric(dataframe):
  print('-------------------------------')
  for column in dataframe.columns:
    try:
      dataframe[column] = dataframe[column].astype(float)
      print(f'[column: {column}] is a numeric set')
    except:
      print(f'[column: {column}] is not a numeric set')
  print('-------------------------------')

In [ ]:
def data_standardization(request_id, dataframe):
  scaler = StandardScaler()
  scaled_features = scaler.fit_transform(dataframe.values)
  dataframe_scaled = pd.DataFrame(scaled_features, columns=dataframe.columns, index=dataframe.index)

  with open(f'/content/drive/MyDrive/Colab Notebooks/Standardizer/model-{request_id}.pkl', 'wb') as file:
      pickle.dump(scaler, file)

  return dataframe_scaled


In [ ]:
def re_standardization(request_id, dataframe):
    with open(f'/content/drive/MyDrive/Colab Notebooks/Standardizer/model-{request_id}.pkl', 'rb') as file:
        scaler = pickle.load(file)

    scaled_features = scaler.transform(dataframe.values)
    dataframe_scaled = pd.DataFrame(scaled_features, columns=dataframe.columns, index=dataframe.index)

    return dataframe_scaled

In [ ]:
def data_normalization(request_id, dataframe):
    scaler = MinMaxScaler()
    normalized_features = scaler.fit_transform(dataframe.values)
    dataframe_normalized = pd.DataFrame(normalized_features, columns=dataframe.columns, index=dataframe.index)

    with open(f'/content/drive/MyDrive/Colab Notebooks/Normalizer/model-{request_id}.pkl', 'wb') as file:
        pickle.dump(scaler, file)

    return dataframe_normalized

In [ ]:
def re_normalization(request_id, dataframe):
  with open(f'/content/drive/MyDrive/Colab Notebooks/Normalizer/model-{request_id}.pkl', 'rb') as file:
      normalizer = pickle.load(file)

  normalized_features = normalizer.inverse_transform(dataframe.values)
  dataframe_normalized = pd.DataFrame(normalized_features, columns=dataframe.columns, index=dataframe.index)

  return dataframe_normalized

In [ ]:
def shuffle_data(dataframe, random_state=42):
    shuffled_data = shuffle(dataframe, random_state=random_state)
    return shuffled_data

### Models

In [ ]:
def train_LSTM(dataframe, y_column_name, window_size=30, unit_size=64):
  # DataFrame
  data = dataframe.copy()

  # KPI column
  kpi_column = data[y_column_name].values.reshape(-1, 1)

  time_steps = window_size  # time steps
  X_train, y_train = LSTM_create_dataset(kpi_column, time_steps)

  # LSTM Create
  model = Sequential()
  model.add(LSTM(unit_size, input_shape=(time_steps, 1)))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')

  # Train
  model.fit(X_train, y_train, epochs=10, batch_size=32)

  return model

In [ ]:
def train_LR(dataframe, y_col_name, fit_intercept=None):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = LinearRegression(fit_intercept=fit_intercept)
  model.fit(X, y)

  return model

In [ ]:
def train_DT(dataframe, y_col_name, max_depth=None, min_samples_split=None):
    y = dataframe.pop(y_col_name).to_numpy()
    X = dataframe.to_numpy()

    model = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)
    model.fit(X, y)

    return model


In [ ]:
def train_RF(dataframe, y_col_name, max_features=None, estimators=None):
    y = dataframe.pop(y_col_name).to_numpy()
    X = dataframe.to_numpy()

    if estimators == None:
      model = RandomForestRegressor(max_features=max_features)
    else:
      model = RandomForestRegressor(max_features=max_features, n_estimators=estimators)

    model.fit(X, y)

    return model

In [ ]:
def train_GB(dataframe, y_col_name, estimators=None, learning_rate=None):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = GradientBoostingRegressor(n_estimators=estimators, learning_rate=learning_rate)
  model.fit(X, y)

  return model


In [ ]:
def train_MLP(dataframe, y_col_name):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = MLPRegressor()
  model.fit(X, y)

  return model

In [ ]:
def train_SVR(dataframe, y_col_name, kernel='linear'):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = SVR(kernel=kernel)
  model.fit(X, y)

  return model

In [ ]:
def train_KNN(dataframe, y_col_name, k=3):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = KNeighborsRegressor(n_neighbors=k)
  model.fit(X, y)

  return model

In [ ]:
def train_BR(dataframe, y_col_name):
  y = dataframe.pop(y_col_name).to_numpy()
  X = dataframe.to_numpy()

  model = BayesianRidge()
  model.fit(X, y)

  return model

### Model training

In [ ]:
def checkTrainRequests():
  request_count = getDataApex(train_request_count_url)[0]['req_count']

  return request_count > 0

In [ ]:
def getTrainRequests():
  return getDataApex(train_request_url)

In [ ]:
def getDataInfo(request_id):
  url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/get-data-and-model-details?id_model={request_id}'

  # read from DB
  rows = getDataApex(url)
  rows = json.dumps(rows)
  data = json.loads(rows)
  df = pd.DataFrame(data)

  # Save dataset
  df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Model-Detail/model-{request_id}.csv', index=False)

  return df

In [ ]:
def getDataset(request_id, model_id, kpi_id, variables, data_size, validate_percent, isShufle = False, isNormalized=False, isStandard=False):

  train_percent = 1-float(validate_percent)
  train_len = math.floor(data_size * train_percent)

  url = train_data_url.replace('"KPI_ID"', f'{kpi_id}')
  url = url.replace('"VARS"', f'{variables}')

  # Get data from DB
  rows = getDataApex(url)
  rows = json.dumps(rows)
  data = json.loads(rows)
  df = pd.DataFrame(data)
  df.set_index('ID', inplace=True)

  # Set header names
  df = df.rename(columns=lambda x: x.replace(" ", "_").upper())

  # Turn numeric
  turn_numeric(df)

  # Encode text column
  encodeText(df, request_id=request_id)

  # Set data size
  df = df[:data_size]

  # Save dataset
  df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/DatabaseSets/model-{request_id}.csv', index=False)

  # Train - Validation Seperation
  train = df[:train_len]
  validation = df[train_len:]

  #if isStandard:
    #train = data_standardization(request_id=request_id, dataframe=train) # Standardization

  #elif isNormalized:
    #train = data_normalization(request_id=request_id, dataframe=train) # Normalization

  #if isShufle:
    #train = shuffle_data(dataframe=train) # Shufle

  return train, validation

In [ ]:
def trainModel(dataframe, model_type, kpi_col_name, model_info):
  kpi_col_name = kpi_col_name.replace(" ", "_").upper()
  train_frame = dataframe.copy()

  if model_type == 1: #  ------------------------ Linear Regresyon
    if model_info['FIT'][0].lower() != 'none':
      intercept = True if (model_info['FIT'][0] == 'TRUE') else False
    else:
      intercept = None

    model = train_LR(train_frame, kpi_col_name, fit_intercept=intercept)

  elif model_type == 2: # ------------------------ Decision Tree Regressor
    if model_info['MIN-SAMPLES-SPLIT'][0].lower() != 'none':
      MSS = int(model_info['MIN-SAMPLES-SPLIT'][0])
    else:
      MSS = 2

    if model_info['MAX-DEPTH'][0].lower() != 'none':
      max_depth = int(model_info['MAX-DEPTH'][0])
    else:
      max_depth = None

    model = train_DT(train_frame, kpi_col_name, max_depth=max_depth, min_samples_split=MSS)

  elif model_type == 3: # Random Forest Regressor
    if model_info['ESTIMATORS'][0].lower() != 'none':
      estimators = int(model_info['ESTIMATORS'][0])
    else:
      estimators = None

    model = train_RF(train_frame, kpi_col_name, estimators=estimators)

  elif model_type == 4: #  ------------------------ Gradient Boosting Regressor
    if model_info['ESTIMATORS'][0].lower() != 'none':
      estimators = int(model_info['ESTIMATORS'][0])
    else:
      estimators = 100

    if model_info['LEARNING-RATE'][0].lower() != 'none':
      l_rate = int(model_info['LEARNING-RATE'][0])
    else:
      l_rate = 0.1

    model = train_GB(train_frame, kpi_col_name, estimators=estimators, learning_rate=l_rate)

  elif model_type == 5: #  ------------------------ Multilayer Perceptron Regressor
    model = train_MLP(train_frame, kpi_col_name)

  elif model_type == 21: #  ------------------------ Support Vector Regression
    model = train_SVR(train_frame, kpi_col_name)

  elif model_type == 22: # ------------------------ K-Neighbors Regressor
    if model_info['K-VALUE'][0].lower() != 'none':
      k_value = int(model_info['K-VALUE'][0])
    else:
      k_value = None

    model = train_KNN(train_frame, kpi_col_name, k=k_value)

  elif model_type == 23: #  ------------------------ Long Short-Term Memory (LSTM) Network

    if model_info['WINDOW_SIZE'][0].lower() != 'none':
      time_step = int(model_info['WINDOW_SIZE'][0])
    else:
      time_step = 5

    if model_info['UNIT-SIZE'][0].lower() != 'none':
      units = int(model_info['UNIT-SIZE'][0])
    else:
      units = 64

    model = train_LSTM(train_frame, kpi_col_name, window_size=time_step, unit_size=units)
  else:
    return None

  return model

### Save Train

In [ ]:
def validateModel(request_id, model_id, model, dataframe, kpi_col_name, model_info=None):
  y_column = kpi_col_name.replace(" ", "_").upper()
  result_frame = dataframe.copy()


  if model_id != 23: # Regression
    X_test = dataframe.copy()
    y_test = X_test.pop(y_column).to_numpy()
    X_test = X_test.to_numpy()
    y_pred = model.predict(X_test)

  else: # LSTM
    if model_info['WINDOW_SIZE'][0] != 'None':
      time_steps = int(model_info['WINDOW_SIZE'][0])
    else:
      time_steps = 5

    kpi_column = dataframe[y_column].values.reshape(-1, 1)
    X_test, y_test = LSTM_create_dataset(kpi_column, time_steps)
    y_pred = model.predict(X_test)

    y_pred = np.squeeze(y_pred)
    y_test = np.squeeze(y_test)

    result_frame = result_frame[time_steps:]

  # Add columns
  result_frame.pop(y_column)
  result_frame['PREDICTION'] = y_pred
  result_frame['ACTUAL'] = y_test
  result_frame['ERROR'] = y_test - y_pred

  # Save dataset
  result_frame.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Validation/model-{request_id}.csv', index=False)

  return result_frame

In [ ]:
def saveVAL(dataframe, request_id):
  val_url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/set-train-val-result?v_kpi_row_id="KPI_ID"&v_model_id={request_id}&v_val="VALUE"'
  i = 0

  val = dataframe.copy()
  val.reset_index(inplace=True)

  for index, row in val.iterrows():
    url = val_url.replace('"KPI_ID"', f"{int(row['ID'])}").replace('"VALUE"', f"{row['PREDICTION']}")
    response = json.loads(postApex(url))

    print(response, end=' ')

In [ ]:
def savePERF(request_id, MAE, MSE, R2, VRNC, error='-'):
  url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/set-model-train-details?id_model_detail={request_id}&v_mse={MSE}&v_mae={MAE}&v_rsv={R2}&v_ev={VRNC}&v_error_message={error}'
  response = postApex(url)
  print(response, end=' ')


In [ ]:
def saveModel(request_id, model):
  path = f"/content/drive/MyDrive/Colab Notebooks/Models/model-{request_id}"

  if model_id != 23:
    with open(f"{path}.pickle", "wb") as file:
      pickle.dump(model, file)
  else:
    model.save(f"{path}.h5")

  return path

### Forecast

In [ ]:
def getForecastRequests():
  url = 'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/get-forcast-requests'

  # Get data from DB
  rows = getDataApex(url)
  rows = json.dumps(rows)
  data = json.loads(rows)
  df = pd.DataFrame(data)

  # Set header names
  df = df.rename(columns=lambda x: x.replace(" ", "_").upper())

  return df

In [ ]:
def getForecastData(request_id, size):
  url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/get-forecast-data?request_id={request_id}'

  # Get data from DB
  rows = getDataApex(url)
  rows = json.dumps(rows)
  data = json.loads(rows)
  df = pd.DataFrame(data)
  df.set_index('ID', inplace=True)

  # Set header names
  df = df.rename(columns=lambda x: x.replace(" ", "_").upper())

  # Turn numeric
  turn_numeric(df)

  return df[:size]

In [ ]:
def forecastData(dataframe, model_id, model, kpi_col_name, time_steps=30):
  kpi_col_name = kpi_col_name.replace(" ", "_").upper()
  result_frame = dataframe.copy()

  if model_id != 23: # Regression
    X_test = dataframe.copy()
    X_test.pop(kpi_col_name)
    X_test = X_test.to_numpy()
    y_pred = model.predict(X_test)

  else: # LSTM
    kpi_column = dataframe.copy()
    kpi_column = kpi_column.pop(kpi_col_name)
    kpi_column = kpi_column.values.reshape(-1, 1)
    X_test, y_test = LSTM_create_dataset(kpi_column, time_steps)
    y_pred = model.predict(X_test)
    result_frame = result_frame[time_steps:]

  # Add columns
  result_frame['PREDICTION'] = y_pred

  # Save dataset
  result_frame.to_csv(f'/content/drive/MyDrive/Colab Notebooks/Forecast/model-{request_id}.csv', index=False)

  return result_frame

In [ ]:
def saveForecast(dataframe, request_id):
  forecast_url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/set-forecast-result?v_request_id={request_id}&v_id_kpi="KPI_ID"&v_value="VALUE"'
  i = 0
  forecast = dataframe.copy()
  forecast.reset_index(inplace=True)

  for index, row in forecast.iterrows():
    url = forecast_url.replace('"KPI_ID"', f"{int(row['ID'])}").replace('"VALUE"', f"{row['PREDICTION']}")
    response = json.loads(postApex(url))

    print(response, end=' ')

In [ ]:
def getKPIname(kpi_id):
  url = f'https://apex.oracle.com/pls/apex/kpi_forecast/Data-Transfer/get-kpi-name?v_id={kpi_id}'
  rows = getDataApex(url)
  return rows[0]['name'].replace(" ", "_").upper()

### Model Performance

In [ ]:
def performance_eval(y_true, y_pred):
  # (Mean Squared Error, MSE)
  mse = mean_squared_error(y_true, y_pred)
  # (Mean Absolute Error, MAE)
  mae = mean_absolute_error(y_true, y_pred)
  # (R-Squared Score)
  r2 = r2_score(y_true, y_pred)
  # (Explained Variance)
  explained_variance = explained_variance_score(y_true, y_pred)

  return mse, mae, r2, explained_variance

In [ ]:
def error_line_plot(y_true, y_pred):
  plt.plot(y_true, label="Actuals")
  plt.plot(y_pred, label="Predictions")
  plt.xlabel("Number of values")
  plt.ylabel("Values")
  plt.title("Comparision of predictions and actuals")
  plt.legend()
  plt.show()

In [ ]:
def error_histogram(y_true, y_pred):
  errors = [true - pred for true, pred in zip(y_true, y_pred)]
  sns.histplot(errors, kde=True)
  plt.xlabel("Error")
  plt.ylabel("Freq")
  plt.title("Error Histogram")
  plt.show()

In [ ]:
def error_distribution_graph(y_true, y_pred):
  errors = [true - pred for true, pred in zip(y_true, y_pred)]
  plt.scatter(y_true, errors)
  plt.axhline(0, color='r', linestyle='--')
  plt.xlabel("Actuals")
  plt.ylabel("Errors")
  plt.title("Error distribution graph")
  plt.show()

In [ ]:
def error_residual_plot(y_true, y_pred):
  residuals = [true - pred for true, pred in zip(y_true, y_pred)]
  plt.scatter(y_pred, residuals)
  plt.axhline(0, color='r', linestyle='--')
  plt.xlabel("Predictions")
  plt.ylabel("Errors")
  plt.title("Error residual plot")
  plt.show()

# Main modules

---



## Train Request Module

---



In [ ]:
trains_json = getTrainRequests()
trains_json = json.dumps(trains_json)
train_requests = pd.read_json(trains_json)

train_requests

,id,model_type,model_name,kpi,kpi_name,train_row_num,validate_percent,id_train_vars
0,201,3,Random Forest Regressor,81,FUEL EFFICIENCY,250,0.15,101:123:102
1,202,21,Support Vector Regression (SVR),81,FUEL EFFICIENCY,300,0.12,121:123:102:104


In [ ]:
for index, request in train_requests.iterrows():
  req_id = request['id']
  model_id = request['model_type']
  model_name = request['model_name']
  kpi_id = request['kpi']
  kpi_name = request['kpi_name']
  variables = request['id_train_vars']
  data_size = request['train_row_num']
  validate_percent = request['validate_percent']

  # Train information
  model_info = getDataInfo(request_id=req_id)
  isNorm = True if model_info['NORMAL'][0].lower() == 'yes' else False
  isStand = True if model_info['STANDARD'][0].lower() == 'yes' else False
  isShufle = True if model_info['SHUFLE'][0].lower() == 'yes' else False

  # Get train dataset
  kpi_train, kpi_val = getDataset(request_id=req_id, model_id=model_id, kpi_id=kpi_id, variables=variables, data_size=data_size, validate_percent=validate_percent, isShufle = isShufle, isNormalized=isNorm, isStandard=isStand)

  # Train and save model
  model = trainModel(dataframe=kpi_train, model_type=model_id, kpi_col_name=kpi_name, model_info=model_info)
  saveModel(request_id=req_id, model=model)

  # Validation
  validation = validateModel(request_id=req_id, model_id=model_id, model=model, dataframe=kpi_val, kpi_col_name=kpi_name, model_info=model_info)

  # True and Pred
  true = validation['ACTUAL'].to_numpy()
  pred = validation['PREDICTION'].to_numpy()

  # Performance
  MSE, MAE, R2, EX_VAR = performance_eval(true, pred)

  # Save validation rows
  saveVAL(validation, req_id)

  # Save validation performance
  savePERF(req_id, MAE, MSE, R2, EX_VAR, error='-')

  # Log
  print(f'\n------ req:{req_id} & kpi:{kpi_name} & model:{model_name} ------')
  display(model_info)
  print('\n------ TRAIN ------')
  display(kpi_train)
  print('\n------ VAL ------')
  display(validation)
  print('\n------ MODEL ------')
  print('MAE: ', MAE)
  print('MSE: ', MSE)
  print('R2 Score: ', R2)
  print('Variance: ', EX_VAR)
  print(model)
  print('\n\n')


-------------------------------
[column: CYLINDERS] is a numeric set
[column: WEIGHT] is a numeric set
[column: ACCELERATION] is a numeric set
[column: FUEL_EFFICIENCY] is a numeric set
-------------------------------
{'saved': 7041} {'saved': 7042} {'saved': 7043} {'saved': 7033} {'saved': 7034} {'saved': 7035} {'saved': 7036} {'saved': 7044} {'saved': 7037} {'saved': 7045} {'saved': 7046} {'saved': 7047} {'saved': 7048} {'saved': 7049} {'saved': 7038} {'saved': 7050} {'saved': 7051} {'saved': 7052} {'saved': 7053} {'saved': 7054} {'saved': 7055} {'saved': 7056} {'saved': 7039} {'saved': 7040} {'saved': 7057} {'saved': 7061} {'saved': 7058} {'saved': 7059} {'saved': 7062} {'saved': 7060} {'saved': 7081} {'saved': 7082} {'saved': 7083} {'saved': 7084} {'saved': 7063} {'saved': 7064} {'saved': 7085} {'saved': 7086} {"saved":129}
 
------ req:201 & kpi:FUEL EFFICIENCY & model:Random Forest Regressor ------


,AUGMENTATION,NULL-DATA-HANDLE,SUM-COLUMN,AVG-COLUMN,NORMAL,STANDARD,SHUFLE,ESTIMATORS,end
0,None,MEDIAN,None,None,NO,NO,YES,NONE,true



------ TRAIN ------


,CYLINDERS,WEIGHT,ACCELERATION,FUEL_EFFICIENCY
ID,,,,
921,8.0,130.0,130.0,18.0
922,8.0,150.0,150.0,16.0
923,8.0,225.0,225.0,14.0
924,8.0,190.0,190.0,15.0
925,8.0,170.0,170.0,15.0
...,...,...,...,...
1128,4.0,70.0,70.0,29.0
1129,4.0,53.0,53.0,33.0
1130,6.0,78.0,78.0,18.0



------ VAL ------


...


------ TRAIN ------


,DISPLACEMENT,ORIGIN,WEIGHT,ACCELERATION,FUEL_EFFICIENCY
ID,,,,,
921,307.0,1.0,130.0,130.0,18.0
922,304.0,1.0,150.0,150.0,16.0
923,455.0,1.0,225.0,225.0,14.0
924,390.0,1.0,190.0,190.0,15.0
925,383.0,1.0,170.0,170.0,15.0
...,...,...,...,...,...
1180,231.0,1.0,165.0,165.0,17.7
1181,225.0,1.0,110.0,110.0,18.6
1182,258.0,1.0,120.0,120.0,18.1



------ VAL ------


,DISPLACEMENT,ORIGIN,WEIGHT,ACCELERATION,PREDICTION,ACTUAL,ERROR
ID,,,,,,,
1185,134.0,3.0,95.0,95.0,25.611396,21.1,-4.511396
1186,151.0,1.0,85.0,85.0,22.357826,23.8,1.442174
1187,119.0,3.0,97.0,97.0,26.070733,23.9,-2.170733
1188,131.0,2.0,103.0,103.0,24.058759,20.3,-3.758759
1189,163.0,2.0,125.0,125.0,22.442001,17.0,-5.442001
1190,121.0,2.0,115.0,115.0,24.106369,21.6,-2.506369
1191,163.0,2.0,133.0,133.0,22.248040,16.2,-6.048040
1192,89.0,2.0,71.0,71.0,26.256519,31.5,5.243481
1193,98.0,3.0,68.0,68.0,27.484800,29.5,2.015200



------ MODEL ------
MAE:  4.0196913661317835
MSE:  26.973345921347736
R2 Score:  0.3184341813349161
Variance:  0.488231552736367
SVR(kernel='linear')





## Forecast Request Module

---



In [ ]:
forecast_requests = getForecastRequests()

forecast_requests

,ID,REQUEST_COMMENT,ID_KPI,CREATION_DATE,CREATED_USER_ID,ID_FORECAST_MODEL,FORECAST_ROW_NUMBER
0,45,Temp request new 1,161,2023-06-14T00:00:00Z,0,164,40


In [ ]:
for index, request in forecast_requests.iterrows():
  request_id = request['ID']
  request_size = request['FORECAST_ROW_NUMBER']
  request_kpi_id = request['ID_KPI']
  forecast_model_id = request['ID_FORECAST_MODEL']
  kpi_name = getKPIname(kpi_id=request_kpi_id)

  try:
    with open(f'/content/drive/MyDrive/Colab Notebooks/Models/model-{forecast_model_id}.pickle', 'rb') as file:
      model = pickle.load(file)
  except:
    with open(f'/content/drive/MyDrive/Colab Notebooks/Models/model-{forecast_model_id}.h', 'rb') as file:
      model = pickle.load(file)

  if forecast_model_id != 23:
    request_data = getForecastData(request_id=request_id, size=request_size)
    forecast_data = forecastData(dataframe=request_data, model_id=forecast_model_id, kpi_col_name=kpi_name, model=model)
  else:
    print('lstm')

  model_info = getDataInfo(request_id=forecast_model_id)

  saveForecast(dataframe=forecast_data, request_id=request_id)

  # Log
  print(f'\n------ Request:{request_id} & kpi:{request_kpi_id} & model:{forecast_model_id} ------')
  print('\n------ REQUEST DATA ------')
  display(request_data)
  print('\n------ FORECAST DATA ------')
  display(forecast_data)
  print('\n------ MODEL ------')
  print(model)
  display(model_info)
  print('\n\n')

-------------------------------
[column: PT08_S2_NMHC] is a numeric set
[column: CO_GT] is a numeric set
[column: DAILY_TEMPERATURE] is a numeric set
-------------------------------
{'saved': 127} {'saved': 155} {'saved': 156} {'saved': 157} {'saved': 158} {'saved': 159} {'saved': 160} {'saved': 161} {'saved': 162} {'saved': 163} {'saved': 128} {'saved': 129} {'saved': 130} {'saved': 131} {'saved': 132} {'saved': 164} {'saved': 165} {'saved': 166} {'saved': 167} {'saved': 133} {'saved': 134} {'saved': 135} {'saved': 136} {'saved': 168} {'saved': 137} {'saved': 169} {'saved': 138} {'saved': 139} {'saved': 140} {'saved': 181} {'saved': 182} {'saved': 183} {'saved': 170} {'saved': 184} {'saved': 171} {'saved': 185} {'saved': 186} {'saved': 172} {'saved': 173} {'saved': 187} 
------ Request:45 & kpi:161 & model:164 ------

------ REQUEST DATA ------


,PT08_S2_NMHC,CO_GT,DAILY_TEMPERATURE
ID,,,
9313,664.500000,-200.0,19.825000
9267,781.000000,-200.0,19.775000
9266,768.000000,-200.0,21.150000
9265,1314.750000,-200.0,23.875000
9264,1037.250000,-200.0,25.425000
9263,962.250000,-200.0,26.450000
9262,928.250000,-200.0,25.724999
9220,1053.000000,-200.0,25.175000
9218,804.250000,-200.0,20.250000



------ FORECAST DATA ------


,PT08_S2_NMHC,CO_GT,DAILY_TEMPERATURE,PREDICTION
ID,,,,
9313,664.500000,-200.0,19.825000,17.086265
9267,781.000000,-200.0,19.775000,19.392152
9266,768.000000,-200.0,21.150000,19.165300
9265,1314.750000,-200.0,23.875000,23.386222
9264,1037.250000,-200.0,25.425000,27.624827
9263,962.250000,-200.0,26.450000,25.301152
9262,928.250000,-200.0,25.724999,20.943099
9220,1053.000000,-200.0,25.175000,28.389363
9218,804.250000,-200.0,20.250000,20.238875



------ MODEL ------
GradientBoostingRegressor()


,AUGMENTATION,NULL-DATA-HANDLE,SUM-COLUMN,AVG-COLUMN,NORMAL,STANDARD,SHUFLE,LEARNING-RATE,ESTIMATORS,end
0,None,AVERAGE,None,None,NO,NO,NO,NONE,NONE,true
